# Day 19

## Part 1

- to crack geodes we need geode-cracking obsidian robots
- to get obsidian we need robots water-proofed with clay
- to get clay we need clay-collecting robots
- all robots need ore, which is gathered by ore-collecitng robots (with big drills)
- I have one of those ore-collecing robots already!
- robots can collect 1 unit of resource per minute
- it takes 1 minute to build a robot but resourses are consumed the instant the robot construction begins
- we have to choose a blueprint (from the input) and stick with it for the run
- a blueprint's quality is its ID number times the number of geodes we can mine using it in `24 minutes`

` What do you get if you add up the quality level of all of the blueprints in your list?`

In [21]:
from copy import deepcopy
from dataclasses import dataclass
from utils import parse_from_file, ParseConfig

@dataclass
class Robot:
    ore: int

@dataclass
class OreRobot(Robot):
    pass

@dataclass
class ClayRobot(Robot):
    pass

@dataclass
class ObsidianRobot(Robot):
    clay: int

@dataclass
class GeodeRobot(Robot):
    obsidian: int

@dataclass
class Blueprint:
    id: int
    ore_bot: OreRobot
    clay_bot: ClayRobot
    obsidian_bot: ObsidianRobot
    geode_bot: GeodeRobot

parser = ParseConfig('\n', ParseConfig(': ', [
    ParseConfig(' ', [None, int]),
    ParseConfig('.', [
        ParseConfig(' ', [None]*4 + [int] + [None]),
        ParseConfig(' ', [None]*5 + [int] + [None]),
        ParseConfig(' ', [None]*5 + [int] + [None]*2 + [int] + [None]),
        ParseConfig(' ', [None]*5 + [int] + [None]*2 + [int] + [None]),
        None
    ])
]))

blueprint_values = parse_from_file(
        'test_inputs\\day_19.txt', parser, unnest_single_items=True)

blueprints = []
for id, (ore, clay, obsidian, geode) in blueprint_values:
    blueprints.append(Blueprint(
        id,
        OreRobot(ore),
        ClayRobot(clay),
        ObsidianRobot(*obsidian),
        GeodeRobot(*geode)
    ))

print(blueprints[0])

Blueprint(id=1, ore_bot=OreRobot(ore=4), clay_bot=ClayRobot(ore=2), obsidian_bot=ObsidianRobot(ore=3, clay=14), geode_bot=GeodeRobot(ore=2, obsidian=7))


In [39]:
@dataclass
class State:
    time_elasped: int
    resources: tuple[int]
    robots: tuple[int]

@dataclass
class Node:
    state: State
    neighbours: dict[State|int]
    visited: bool = False

def get_next_neighbours(
    blueprint: Blueprint, node: Node , total_time: int
) -> list[tuple[int]]:
    """
    returns a list of neighbours to the current node
    """
    ore, clay, obsidian, geode = node.state.resources
    ore_bots, clay_bots, obsidian_bots, geode_bots = node.state.robots
    new_ore_bots, new_clay_bots, new_obsidian_bots, new_geode_bots = 0, 0, 0, 0
    ore_neighbour, clay_neighbour, obsidian_neighbour, geode_neighbour = \
        False, False, False, False
    neighbours = []
    for time_delta in range(total_time - node.state.time_elasped):
        # print(new_ore_bots, new_clay_bots, new_obsidian_bots, new_geode_bots)
        robot = blueprint.geode_bot
        if (
            ore >= robot.ore and
            obsidian >= robot.obsidian and
            new_geode_bots == 0
        ):
            ore -= robot.ore
            obsidian -= robot.obsidian
            new_geode_bots += 1
        
        robot = blueprint.obsidian_bot
        if (
            ore >= robot.ore and
            clay >= robot.clay and
            new_obsidian_bots == 0
        ):
            ore -= robot.ore
            clay -= robot.clay
            new_obsidian_bots += 1
        
        robot = blueprint.clay_bot
        if ore >= robot.ore and new_clay_bots == 0:
            new_clay_bots += 1
        
        robot = blueprint.ore_bot
        if ore >= robot.ore and new_ore_bots == 0:
            new_ore_bots += 1

        # let's go mining!
        ore += ore_bots
        clay += clay_bots
        obsidian += obsidian_bots
        geode += geode_bots

        # check if we have a new neighbour
        if new_geode_bots > 0 and not geode_neighbour:
            robot = blueprint.geode_bot
            neighbours.append(State(
                node.state.time_elasped + time_delta + 1,
                (ore - robot.ore, clay, obsidian - robot.obsidian, geode),
                (ore_bots, clay_bots, obsidian_bots, geode_bots + 1)
            ))
            geode_neighbour = True
        if new_obsidian_bots > 0 and not obsidian_neighbour:
            robot = blueprint.obsidian_bot
            neighbours.append(State(
                node.state.time_elasped + time_delta + 1,
                (ore - robot.ore, clay - robot.clay, obsidian, geode),
                (ore_bots, clay_bots, obsidian_bots + 1, geode_bots)
            ))
            obsidian_neighbour = True
        if new_clay_bots > 0 and not clay_neighbour:
            robot = blueprint.clay_bot
            neighbours.append(State(
                node.state.time_elasped + time_delta + 1,
                (ore - robot.ore, clay, obsidian, geode),
                (ore_bots, clay_bots + 1, obsidian_bots, geode_bots)
            ))
            clay_neighbour = True
        if new_ore_bots > 0 and not ore_neighbour:
            robot = blueprint.ore_bot
            neighbours.append(State(
                node.state.time_elasped + time_delta + 1,
                (ore - robot.ore, clay, obsidian, geode),
                (ore_bots + 1, clay_bots, obsidian_bots, geode_bots)
            ))
            ore_neighbour = True
        if all([
            new_bots > 0 for new_bots in \
            (ore_bots, clay_bots, obsidian_bots, geode_bots)
        ]):
            break

    return neighbours

test_node = Node(State(0, (0, 0, 0, 0), (1, 0, 0, 0)), [])
print(blueprints[0])
print(get_next_neighbours(blueprints[0], test_node, 24))

Blueprint(id=1, ore_bot=OreRobot(ore=4), clay_bot=ClayRobot(ore=2), obsidian_bot=ObsidianRobot(ore=3, clay=14), geode_bot=GeodeRobot(ore=2, obsidian=7))
[State(time_elasped=3, resources=(1, 0, 0, 0), robots=(1, 1, 0, 0)), State(time_elasped=7, resources=(1, 0, 0, 0), robots=(2, 0, 0, 0))]


In [ ]:
# let's go full n dimensional!
def dijkstra(
        grid: dict[Node], start: tuple[tuple[int]],
        end: 'tuple[tuple[int]]|None'
) -> 'list[list[int]]':
    """
    run Djikstra's algorithm to find the shortest distance from start to end
    """
    # mark all nodes as unvisited
    visited = [[[False for _ in row] for row in z_slice] for z_slice in grid]

    # set all notes to a tentative max distance from start
    max_distance = len(grid) * len(grid[0]) * len(grid[0][0])
    distances = [
        [[max_distance for _ in row] for row in z_slice] for z_slice in grid
    ]
    # ...except for our starting node which is 0s
    x, y, z = start
    distances[z][y][x] = 0

    # set current node as the start
    current_node = start

    number_visited = 0
    while True:
        x, y, z = current_node
        # iterate through each neighbour and set it's new tentative distance
        distance_to_here = distances[z][y][x]
        for neighbour in net[current_node]:
            x, y, z = neighbour
            if visited[z][y][x]:
                continue
            distances[z][y][x] = min(distances[z][y][x], distance_to_here + 1)
        x, y, z = current_node
        # mark this node as visited
        visited[z][y][x] = True
        number_visited += 1

        # if this node is the end, we're done!
        if current_node == end:
            return distances
        if number_visited % 100 == 0:
            percent_done = 100 * number_visited / max_distance
            print(f'{current_node} - {percent_done:.1f}%' + ' '*10, end='\r')
        
        # otherwise find the node with the smallest tenative distance and
        # repeat
        ux, uy, uz = None, None, None
        lowest_unvisited = max_distance
        for z, z_slice in enumerate(visited):
            for y, row in enumerate(z_slice):
                for x, is_visited in enumerate(row):
                    if is_visited:
                        continue
                    if distances[z][y][x] < lowest_unvisited:
                        ux, uy, uz = x, y, z
                        lowest_unvisited = distances[z][y][x]
        if lowest_unvisited == max_distance:
            # if true all reachable points have been visited
            return distances
        current_node = (ux, uy, uz)

outer_distances = dijkstra(grid, net, (0, 0, 0), None)